In [ ]:
"""This code demonstrates how to preprocess a dataset, create dummy variables for categorical features, 
and train a neural network model using TensorFlow to predict energy consumption per hour.
"""

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [33]:
"""
Data Preprocessing
"""
# Loading data
data = pd.read_csv("finaldata.csv")  # Read the data from a CSV file

# Convert the 'DATE' column to datetime format
data["DATE"] = pd.to_datetime(data["DATE"], format="%d/%m/%Y")

# Set the 'DATE' column as index and reset the index
data.set_index("DATE", inplace=True)
data.reset_index(inplace=True)

# Create new columns for date, id, hour, day_of_week, and energy_per_hour
data["date"] = data["DATE"]
data["id"] = data["LCLid"]
data['hour'] = data["HOUR"]
data['day_of_week'] = data["DATE"].dt.day_of_week
data['energy_per_hour'] = data["ENERGY.PER.HOUR"]

# Create dummy variables for 'day_of_week' and concatenate with the original DataFrame
dummies = pd.get_dummies(data['day_of_week'], prefix='day_of_week')
data = pd.concat([data, dummies], axis=1)

# Create dummy variables for 'hour' and concatenate with the original DataFrame
dummies = pd.get_dummies(data['hour'], prefix='hour')
data = pd.concat([data, dummies], axis=1)

# Filter the data to only include records with id 'MAC000002'
data['year'] = data['date'].dt.year
data = data.query("id == 'MAC000002'")

# Reset the index and sort the data by date and hour
data.reset_index(inplace=True)
data = data.drop(columns=["index"])
data["date"] = pd.to_datetime(data["date"])
data = data.sort_values(by=["date", "hour"]).reset_index(drop=True)

# Drop unnecessary columns
data = data.drop(columns=["DATE", "LCLid", "HOUR", "ENERGY.PER.HOUR", "id", "day_of_week", "hour"])
data = data.drop(columns=["X", "MINUTES", "SECONDS", "precipType", "icon", "summary", "Acorn", "year"])
data = data.drop(columns=["date"])


In [34]:
# Prepare input (X) and output (y) data for the model
X = data.drop(columns=['energy_per_hour'])
y = data['energy_per_hour'].astype(float)

In [29]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale input data using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the three layer neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

# Train the model
model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)

# Make predictions on the test set
y_pred_nn = model.predict(X_test_scaled)

# Calculate the Root Mean Squared Error (RMSE)
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))

print("Neural Network RMSE:", rmse_nn)

Epoch 1/100


/opt/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 0.1677 - val_loss: 0.1831
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1503 - val_loss: 0.1812
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1464 - val_loss: 0.1807
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1435 - val_loss: 0.1806
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1419 - val_loss: 0.1834
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1402 - val_loss: 0.1805
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1397 - val_loss: 0.1800
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1380 - val_loss: 0.1821
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1373 - val_loss: 0.1819
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.1368 - val_loss: 0.1821
E

In [35]:
# Save the trained model to an HDF5 file
model.save('production_model.h5')